In [ ]:
"""
How started running the meshParty

pip3 install meshparty
git clone https://github.com/sdorkenw/MeshParty.git
cd MeshParty
pip install . --upgrade

"""

In [1]:
from os import sys
# sys.path.append("../../meshAfterParty/")
sys.path.append("../../meshAfterParty/")
#sys.path.append("../../MeshParty/")
from importlib import reload
import os
from pathlib import Path
os.getcwd()


import neuron_utils as nru
nru = reload(nru)
import neuron
neuron=reload(neuron)

import system_utils as su

In [7]:
dir(meshparty)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__']

# Getting the Neuron to process

In [2]:
compressed_neuron_path = Path("../test_neurons/test_objects/12345_2_soma_practice_decompress")


with su.suppress_stdout_stderr():
    recovered_neuron = nru.decompress_neuron(filepath=compressed_neuron_path,
                      original_mesh=compressed_neuron_path)

recovered_neuron

In [3]:
import skeleton_utils as sk
current_limb = recovered_neuron.concept_network.nodes["L1"]["data"]
sk.graph_skeleton_and_mesh(other_meshes=[current_limb.mesh])

In [8]:
from meshparty import trimesh_io

In [9]:
limb_object = trimesh_io.Mesh(vertices = current_limb.mesh.vertices,
                                       faces = current_limb.mesh.faces,
                                       normals=current_limb.mesh.face_normals)

In [10]:
import meshparty.skeletonize as skel


limb_skeleton = skel.skeletonize_mesh(limb_object, soma_pt=None, soma_radius=7500, collapse_soma=False,
                     invalidation_d=12000, smooth_vertices=False, compute_radius=False,
                     compute_original_index=True, cc_vertex_thresh=100, verbose=True)

extracted_skeleton = limb_skeleton.vertices[limb_skeleton.edges]


100%|██████████| 1/1 [00:01<00:00,  1.51s/it]


# Skeleton Tutorial

# --- mesh_to_skel: mesh vertices to skeleton vertices

In [20]:
# the mesh_to_skel mapping (maps original vertices to skeleton vertices)
limb_skeleton.mesh_to_skel_map.shape, limb_object.vertices.shape

((87663,), (87663, 3))

# -- segments: list of skeleton vertices belonging to one branch

In [26]:
#The limb segments show that the skeleton vertices are broken down into branches if need be
import numpy as np
np.concatenate(limb_skeleton.segments).shape, limb_skeleton.vertices.shape, 

((5691,), (5691, 3), 45)

# -- segment_map: skeleton vertices to segment

In [27]:
#The limb segments show that the skeleton vertices are broken down into branches if need be
import numpy as np
limb_skeleton.segment_map.shape, np.max(limb_skeleton.segment_map), len(limb_skeleton.segments)

((5691,), 44, 45)

# Working with root node

In [39]:
returned_sparse_matrix = meshparty.utils.create_csgraph(limb_skeleton.vertices,limb_skeleton.edges,
                              euclidean_weight = True,
                              directed=True)

returned_sparse_matrix.csgraph

AttributeError: csgraph not found

In [41]:
import scipy
dsn,pred = scipy.sparse.csgraph.dijkstra(limb_skeleton.csgraph,return_predecessors=True)
dsn.shape, limb_skeleton.vertices.shape

5

# Visualizing the skeleton endpoints and branch points

array([[ 820653.2, 1006871. ,  862538.4],
       [ 829318.6, 1011721. ,  855262.4],
       [ 846551.4,  998493.9,  862480.3],
       [ 815535.6, 1002435. ,  865409.1],
       [ 777495.2, 1027902. ,  844597.4],
       [ 653983.8, 1066320. ,  847478.2],
       [ 808532.8,  969424.8,  863596.8],
       [ 751811.2, 1034690. ,  841723.8],
       [ 865978.4, 1037040. ,  830443.1],
       [ 810883.8, 1017888. ,  850808.9],
       [ 812447.2, 1017466. ,  851035.6],
       [ 834785.9, 1008575. ,  858022.4],
       [ 740048.9, 1036667. ,  840557.6],
       [ 866218.3, 1034524. ,  829425.6],
       [ 844418.5, 1000885. ,  861691.6],
       [ 795976.5, 1005431. ,  865267.1],
       [ 740926.5, 1038806. ,  839698.2],
       [ 621561.1, 1079942. ,  847227.7],
       [ 812805.9, 1017162. ,  850446. ],
       [ 783709.7,  967298.2,  880497. ],
       [ 819636.3, 1006324. ,  862835.8],
       [ 812673.1, 1017186. ,  850520.1]])

In [91]:
curr_dist_to_root = limb_skeleton.distance_to_root[limb_skeleton.end_points]
np.argsort(curr_dist_to_root)

array([ 1, 21,  0,  4, 20, 14, 18,  8, 11, 22,  5,  9,  2, 10, 19,  3, 16,
       12, 15,  7,  6, 17, 13])

In [92]:
curr_dist_to_root[1], curr_dist_to_root[0]

(101500.21876525879, 239378.97985458374)

In [95]:
limb_skeleton.parent_node(1)#limb_skeleton.root)

2472

In [93]:
limb_skeleton.root

5240

In [83]:
sk.graph_skeleton_and_mesh(
    other_scatter=[limb_skeleton.vertices[limb_skeleton.branch_points],
                  limb_skeleton.vertices[limb_skeleton.end_points],
                   limb_skeleton.vertices[limb_skeleton.root].reshape(-1,3)
                  ],
    scatter_size=5,
    other_scatter_colors=["red","yellow","pink"],
     other_skeletons=[extracted_skeleton]
)

# -- cut branches and downstream nodes

In [96]:
limb_skeleton.root

5240

In [107]:
2922
current_path_to_root = limb_skeleton.path_to_root(1)
list(current_path_to_root)

[1,
 2472,
 844,
 4615,
 2450,
 2029,
 4462,
 5056,
 187,
 384,
 3121,
 4803,
 2267,
 1984,
 3018,
 4741,
 1317,
 522,
 2643,
 3774,
 2490,
 740,
 33,
 99,
 5676,
 1433,
 3997,
 1153,
 5177,
 1092,
 4098,
 103,
 5574,
 1289,
 182,
 195,
 5191,
 2631,
 2232,
 1666,
 3403,
 4437,
 4334,
 2923,
 5491,
 5211,
 658,
 4623,
 555,
 4925,
 616,
 1634,
 3072,
 1898,
 1864,
 1067,
 265,
 1025,
 4876,
 1871,
 289,
 3201,
 904,
 2236,
 2284,
 5462,
 965,
 2276,
 1388,
 946,
 776,
 372,
 1573,
 3295,
 2245,
 1269,
 4464,
 2067,
 4126,
 908,
 3673,
 3813,
 1105,
 1912,
 5017,
 218,
 4354,
 1659,
 1445,
 2005,
 461,
 3219,
 1730,
 2293,
 3249,
 472,
 3715,
 192,
 1305,
 4046,
 1923,
 1399,
 757,
 3388,
 2844,
 1852,
 1233,
 944,
 394,
 1166,
 858,
 5439,
 4495,
 4743,
 1273,
 5527,
 1234,
 1229,
 4279,
 3348,
 4753,
 3282,
 2813,
 1888,
 5232,
 5509,
 35,
 55,
 1684,
 3493,
 3776,
 1747,
 2724,
 5401,
 3828,
 1031,
 3044,
 982,
 2878,
 4032,
 4982,
 994,
 2168,
 4127,
 1356,
 2967,
 4304,
 5608,
 258

In [111]:
limb_skeleton.end_points

array([ 477,  781, 1222, 1277, 1442, 1588, 1775, 1810, 2020, 2368, 2911,
       3117, 3343, 3807, 3920, 3928, 3984, 4483, 4525, 4773, 4879, 5025,
       5169])

In [123]:
limb_skeleton.downstream_nodes(5424)

array([ 102,  184,  293,  297,  303,  332,  339,  352,  402,  431,  440,
        519,  605,  653,  675,  733,  842,  865, 1161, 1171, 1277, 1292,
       1296, 1336, 1365, 1367, 1402, 1449, 1498, 1521, 1537, 1559, 1579,
       1657, 1662, 1728, 1838, 1873, 1879, 1906, 2009, 2013, 2039, 2071,
       2113, 2125, 2134, 2195, 2219, 2246, 2262, 2286, 2301, 2313, 2373,
       2385, 2475, 2494, 2530, 2552, 2562, 2621, 2689, 2707, 2781, 2791,
       2809, 2836, 2839, 2883, 2943, 2961, 3013, 3142, 3177, 3247, 3407,
       3446, 3515, 3528, 3570, 3571, 3830, 3929, 3943, 3983, 4096, 4116,
       4208, 4209, 4280, 4311, 4356, 4387, 4608, 4703, 4722, 4725, 4927,
       4986, 5032, 5109, 5117, 5130, 5146, 5424, 5435, 5443, 5474, 5647,
       5660, 5680])

In [ ]:
limb_skeleton

In [128]:
limb_skeleton.csgraph[477,5455],limb_skeleton.csgraph[5455,477]

(111.48408, 0.0)

In [126]:
limb_skeleton.edges[np.where(limb_skeleton.edges[:,0] == 477)[0]]

array([[ 477, 5455]])

In [129]:
limb_skeleton.edges.shape

(5690, 2)

In [130]:
limb_skeleton.vertices.shape

(5691, 3)

# Working on Skeletonize

In [134]:
np.linalg.norm(np.random.rand(9,3),axis=1)

array([0.78268469, 0.53294745, 0.82264316, 1.40793313, 1.04565141,
       1.07131435, 0.88430512, 1.03756003, 1.06038901])

In [135]:
from scipy import sparse

In [150]:
N = 5
eye = sparse.csc_matrix((np.ones(N,dtype=np.float32),(np.arange(0,N),np.arange(0,N))),shape=(N,N))

In [151]:
eye.toarray()

array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.]], dtype=float32)

In [157]:
C = sparse.csc_matrix(eye)
CA = C.toarray()

In [156]:
# defining the connectivity matrix
A = np.array([[0,1,0,0,0],
             [0,0,1,0,0],
             [0,0,0,1,1],
              [0,0,0,0,0],
             [0,0,0,0,0]])
S = A + A.T
S 

In [163]:
np.linalg.matrix_power(S,2)

array([[1, 0, 1, 0, 0],
       [0, 2, 0, 1, 1],
       [1, 0, 3, 0, 0],
       [0, 1, 0, 1, 1],
       [0, 1, 0, 1, 1]])

In [165]:
np.linalg.matrix_power(S + CA,1)

array([[1., 1., 0., 0., 0.],
       [1., 1., 1., 0., 0.],
       [0., 1., 1., 1., 1.],
       [0., 0., 1., 1., 0.],
       [0., 0., 1., 0., 1.]])

In [161]:
np.linalg.matrix_power(S + CA,2)

array([[2., 2., 1., 0., 0.],
       [2., 3., 2., 1., 1.],
       [1., 2., 4., 2., 2.],
       [0., 1., 2., 2., 1.],
       [0., 1., 2., 1., 2.]])

In [166]:
D = np.linalg.matrix_power(S + CA,3)
D

array([[ 4.,  5.,  3.,  1.,  1.],
       [ 5.,  7.,  7.,  3.,  3.],
       [ 3.,  7., 10.,  6.,  6.],
       [ 1.,  3.,  6.,  4.,  3.],
       [ 1.,  3.,  6.,  3.,  4.]])

In [169]:
D = sparse.csc_matrix(D)
D.sign().toarray()

array([[1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.]])

In [177]:
C_current = sparse.csc_matrix(np.linalg.matrix_power(S + CA,2))
C_current = C_current.sign()
neighbors = np.sum(C_current,axis = 1)
neighbors = 1/neighbors
print(C_current.toarray(),neighbors)

[[1. 1. 1. 0. 0.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [0. 1. 1. 1. 1.]
 [0. 1. 1. 1. 1.]] [[0.33333333]
 [0.2       ]
 [0.2       ]
 [0.25      ]
 [0.25      ]]


In [189]:
C_adjusted = C_current.multiply(neighbors)
type(C_adjusted)
C_adjusted*sparse.csc_matrix(np.random.rand(5,3)).toarray()

array([[0.59504358, 0.59942038, 0.33271531],
       [0.62176768, 0.58090391, 0.40698172],
       [0.62176768, 0.58090391, 0.40698172],
       [0.58005193, 0.63055384, 0.398194  ],
       [0.58005193, 0.63055384, 0.398194  ]])

In [183]:
type(C_current)

scipy.sparse.csc.csc_matrix

In [190]:
time_per_job = 
90000 * 

0.96